In [3]:
from albumentations import (
    HorizontalFlip, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine, Flip, OneOf, Compose
)

import pandas as pd
import os
from tqdm import tqdm
import re
from numpy import nan as Nan
import pydicom as dicom
import cv2
import shutil


In [4]:
def get_label_for_patient(lad_segment, reports, patient_id):
    """
    
    Example:
        get_label_for_patient('LAD', reports, 'DDJ261Z' )

    """
    classes_to_positive = ['<25%','25%', 'NORMAL', '-']

    if lad_segment == 'LAD':
        curr_section_label = reports.loc[reports['PATIENT_ID'] == patient_id][['MID', 'PROXIMAL', 'DISTAL']].iloc[0]
        stenosis_score = [x for x in curr_section_label if x not in classes_to_positive]
        label = 1 if len(stenosis_score) > 0 else 0
        return label, '___'.join(curr_section_label.values)
        
    else:
        curr_section_label =  reports.loc[reports['PATIENT_ID'] == patient_id][lad_segment].iloc[0]
        label = 0 if curr_section_label in classes_to_positive else 1    
        return label, curr_section_label

def get_labels(path_to_patient, reports):
    """
    Returns:
        - dict: key(str) - type of the artery, value(int) - label(0 or 1)
    """
    
    types_of_images = os.listdir(path_to_patient)
    patient_name = [x for x in path_to_patient.split('/') if len(x) > 0][-1]
    labels_dict = {}
    
    for i in range(len(types_of_images)):
        labels_dict[types_of_images[i]] = get_label_for_patient(types_of_images[i], reports, patient_name)
        
    return labels_dict

def get_imgs_names_to_the_labels(path_to_patient, labels_dict):
    """
    Args:
        - dict: key - branch of artery(str), value - label(int) 0 or 1.   
    Returns:
        - dict: key - label(int) 0 or 1, value(list) list of images, belong to the labelT
    """
    img_labels = {}

    for key in labels_dict.keys():
        list_of_images = os.listdir(os.path.join(path_to_patient, key))
        if labels_dict[key][1]=='-' or labels_dict[key][1]=='-___-___-':
            continue
        if labels_dict[key] in img_labels:
            img_labels[labels_dict[key]]+= [x for x in list_of_images if '_text_deleted' in x]
        else:
            img_labels[labels_dict[key]] = [x for x in list_of_images if '_text_deleted' in x]
            
    return img_labels




In [6]:
reports = pd.read_excel('lad_reports.xlsx')
reports.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'lad_reports.xlsx'

In [5]:
PATH_TO_DATA = r'/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_only_lad/train'
list_of_patients = os.listdir(PATH_TO_DATA)
means = []
labels_csv = pd.DataFrame(columns=['PATIENT_NAME', 'IMG_NAME', 'LABEL', 'STENOSIS_SCORE'])

for i in tqdm(range(len(list_of_patients))):
    if i == 0:
        continue
    labels = get_labels(os.path.join(PATH_TO_DATA, list_of_patients[i]), reports)
    dict_labels_images = get_imgs_names_to_the_labels(os.path.join(PATH_TO_DATA, list_of_patients[i]), labels)
    for key in dict_labels_images:
        for j in range(len(dict_labels_images[key])):
            new_row = pd.Series([list_of_patients[i],dict_labels_images[key][j], key[0], key[1]],index=labels_csv.columns)
            img  = cv2.imread(PATH_TO_DATA, list_of_patients[i], dict_labels_images[key][j])
            break
        break
    break

  0%|          | 0/694 [00:00<?, ?it/s]


NameError: name 'reports' is not defined